In [1]:
# FIRST ATTEMPT AT OPTIMIZATION

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' and AFFILIATION/ STATUS/ ASK_AMT columns
columns_to_drop = ['EIN', 'NAME', 'AFFILIATION', 'STATUS', 'ASK_AMT']
application_df.drop(columns=columns_to_drop, inplace=True)
application_df.head(10)

In [ ]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

In [ ]:
print(application_df.value_counts())

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_counts = application_df["APPLICATION_TYPE"].value_counts()
app_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classy_counts = application_df["CLASSIFICATION"].value_counts()
classy_counts 

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
helpful_classy_counts = classy_counts.loc[classy_counts > 1]
helpful_classy_counts[:15]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classy_counts[classy_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
app_numeric = pd.get_dummies(application_df)
app_numeric

In [ ]:
# Split our preprocessed data into our features and target arrays
X = app_numeric.drop('IS_SUCCESSFUL', axis=1)
y = app_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
# Working in google colab so the code gets interesting here
# import os dependency
import os
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
file_path = os.path.join(desktop_path, "AlphabetSoupCharity.h5")
nn.save(file_path)

# save file from google colab
from google.colab import files
files.download(file_path)
# Ensure file was saved into correct directory
print(f"Model saved to {file_path}")

In [ ]:
# This attempt at optimization did NOT work the accuracy score went down to 65%